In [0]:
df = spark.read.option("multiline","true").json("/FileStore/Datasets/JSON files/PT1H.json")

display(df)

EventName,body,category,correlationId,end,env_dt_spanId,env_dt_traceId,env_name,env_time,env_ver,groupId,level,location,name,operationName,pipelineName,properties,resourceId,runId,severityNumber,severityText,start,status,tags,time,timestamp
SynapseDiagnosticsLogs,"System.Collections.Generic.List`1[System.Collections.Generic.KeyValuePair`2[System.String,System.Object]]",IntegrationPipelineRuns,5f253516-93be-42dc-a563-293f6412377a,1601-01-01T00:00:00.0000000Z,9f8e6872ca93ec9d,5b87a0ae9dcc7d21ce1b48be3f67772c,SynapseDiagnosticsLogsSourceSynapse,2024-03-25T13:00:05.2430190Z,4.0,5f253516-93be-42dc-a563-293f6412377a,Informational,westus,SynapseDiagnosticsLogs,BI_HR_Adhoc_Pipeline - InProgress,BI_HR_Adhoc_Pipeline,"List(List(), , List(List(08584902352849187542608497116CU135, ScheduleTrigger, BI_HR_Adhoc_Pipeline)), List(2024-03-25T13:00:05.1583905Z, 2024-03-25T13:00:02.7786512+00:00, 0a1a0520-a1e4-4774-843e-c59cf6cf490a, 08584902352849187542608497116CU135))",/SUBSCRIPTIONS/0A1A0520-A1E4-4774-843E-C59CF6CF490A/RESOURCEGROUPS/UGNX_SYNAPSE-RG/PROVIDERS/MICROSOFT.SYNAPSE/WORKSPACES/UGNX-SYNAPSE-DW,5f253516-93be-42dc-a563-293f6412377a,9,Information,2024-03-25T13:00:02.0000000Z,InProgress,"{""Cost Center"":""817-Data Management""}",2024-03-25T13:00:05.2430190Z,2024-03-25T13:00:05.0000000Z


In [0]:


from pyspark.sql.types import *
from pyspark.sql.functions import *

#Flatten array of structs and structs
def flatten(df):
   #compute Complex Fields (Lists and Structs) in Schema   
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))
    
      #if StructType then convert all sub element to columns.
      #i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)
    
      #if ArrayType then add the Array Elements as Rows using the explode function
      #i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):    
         df=df.withColumn(col_name,explode_outer(col_name))
    
      #recompute remaining Complex Fields in Schema       
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df

In [0]:
df_flatten = flatten(df)

display(df_flatten)

Processing :properties Type : <class 'pyspark.sql.types.StructType'>
Processing :properties_Annotations Type : <class 'pyspark.sql.types.ArrayType'>
Processing :properties_Predecessors Type : <class 'pyspark.sql.types.ArrayType'>
Processing :properties_Predecessors Type : <class 'pyspark.sql.types.StructType'>
Processing :properties_SystemParameters Type : <class 'pyspark.sql.types.StructType'>


EventName,body,category,correlationId,end,env_dt_spanId,env_dt_traceId,env_name,env_time,env_ver,groupId,level,location,name,operationName,pipelineName,resourceId,runId,severityNumber,severityText,start,status,tags,time,timestamp,properties_Annotations,properties_Message,properties_Predecessors_Id,properties_Predecessors_InvokedByType,properties_Predecessors_Name,properties_SystemParameters_ExecutionStart,properties_SystemParameters_PipelineRunRequestTime,properties_SystemParameters_SubscriptionId,properties_SystemParameters_TriggerId
SynapseDiagnosticsLogs,"System.Collections.Generic.List`1[System.Collections.Generic.KeyValuePair`2[System.String,System.Object]]",IntegrationPipelineRuns,5f253516-93be-42dc-a563-293f6412377a,1601-01-01T00:00:00.0000000Z,9f8e6872ca93ec9d,5b87a0ae9dcc7d21ce1b48be3f67772c,SynapseDiagnosticsLogsSourceSynapse,2024-03-25T13:00:05.2430190Z,4.0,5f253516-93be-42dc-a563-293f6412377a,Informational,westus,SynapseDiagnosticsLogs,BI_HR_Adhoc_Pipeline - InProgress,BI_HR_Adhoc_Pipeline,/SUBSCRIPTIONS/0A1A0520-A1E4-4774-843E-C59CF6CF490A/RESOURCEGROUPS/UGNX_SYNAPSE-RG/PROVIDERS/MICROSOFT.SYNAPSE/WORKSPACES/UGNX-SYNAPSE-DW,5f253516-93be-42dc-a563-293f6412377a,9,Information,2024-03-25T13:00:02.0000000Z,InProgress,"{""Cost Center"":""817-Data Management""}",2024-03-25T13:00:05.2430190Z,2024-03-25T13:00:05.0000000Z,null,,08584902352849187542608497116CU135,ScheduleTrigger,BI_HR_Adhoc_Pipeline,2024-03-25T13:00:05.1583905Z,2024-03-25T13:00:02.7786512+00:00,0a1a0520-a1e4-4774-843e-c59cf6cf490a,08584902352849187542608497116CU135
